In [20]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]


In [2]:
import openai as client

# assistant = client.beta.assistants.create(
#   name="Investor Assistant",
#   instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#   model="gpt-4-1106-preview",
#   tools=functions
# )

assistant_id = "asst_sNRfGqBlfUQnarB6j5clyGxW"

In [3]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_9zrU6pX7oIo8xPr3vBYyEnfe', created_at=1712913888, metadata={}, object='thread')

In [24]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_N8RrgxXBgWIYCwUFno5CiToV', assistant_id='asst_sNRfGqBlfUQnarB6j5clyGxW', cancelled_at=None, completed_at=None, created_at=1712915938, expires_at=1712916538, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_9zrU6pX7oIo8xPr3vBYyEnfe', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type':

In [28]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"calling function: {function.name} with arg {function.arguments}")
        outputs.append({
            "output": functions_map[function.name](json.loads(function.arguments)),
            "tool_call_id": action_id
        })
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs
    )

In [33]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
user: please go ahead
assistant: I just sent the command to retrieve the ticker symbol for Salesforce. As soon as I receive the ticker information, I will be able to gather more data about the company's financials and recent stock performance to help you with your decision. Please hold on for a moment while I process this information.
assistant: I have received the ticker symbol for Salesforce, which is "CRM". I will now gather the relevant financial data and stock performance for this company. We will look at Salesforce's income statement, balance sheet, and its stock performance over the last 100 days to assess whether it might be a good investment.

Please give me a moment to retrieve the necessary information.
assistant: Based on the financial data and stock performance from the last 100 days, we have the following information about Salesforce (CRM):

### Income Statement Highlights (Most Recent Data Available)
- Total Reve

In [29]:
submit_tool_outputs(run.id, thread.id)

calling function: get_income_statement with arg {"ticker": "CRM"}
calling function: get_balance_sheet with arg {"ticker": "CRM"}
calling function: get_daily_stock_performance with arg {"ticker": "CRM"}


Run(id='run_N8RrgxXBgWIYCwUFno5CiToV', assistant_id='asst_sNRfGqBlfUQnarB6j5clyGxW', cancelled_at=None, completed_at=None, created_at=1712915938, expires_at=1712916538, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1712915939, status='queued', thread_id='thread_9zrU6pX7oIo8xPr3vBYyEnfe', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'

In [32]:
get_run(run.id, thread.id).status

'completed'